# Step 1: Build DateDim table

In [0]:
%sql
-- Extract min and max date
SELECT
  CAST(min(order_date) AS DATE)  AS d_min,
  CAST(max(COALESCE(ship_date, order_date)) AS DATE) AS d_max
FROM main.retail.superstore_silver;


## Creating DateDim

In [0]:
%sql

create or replace table main.retail_gold.DateDim (

  -- Core, non-generated columns
  date DATE NOT NULL
    COMMENT 'Calendar date (day grain)',
  date_sk INT NOT NULL GENERATED ALWAYS AS (CAST(date_format(date,'yyyyMMdd') AS INT))
    COMMENT 'Surrogate key YYYYMMDD, generated from date',
  
  -- Generated columns from date
  day_of_week TINYINT NOT NULL GENERATED ALWAYS AS (
    CAST(pmod(dayofweek(date) + 5, 7) + 1 AS TINYINT)
    )
    COMMENT '1=Mon,..., 7=Sun (ISO)',
  
  dow_name STRING NOT NULL GENERATED ALWAYS AS (date_format(date,'EEE'))
    COMMENT 'Mon, Tue,..., Sun (ISO)',
  
  week SMALLINT NOT NULL  GENERATED ALWAYS AS (
    CAST(weekofyear(date) AS SMALLINT)
    )
    COMMENT 'Week number 1 - 53',
  
  month tinyint NOT NULL GENERATED ALWAYS AS (
    CAST(month(date) AS TINYINT)
    )
    COMMENT 'month number 1-12',
  
  month_name STRING NOT NULL GENERATED ALWAYS AS (date_format(date,'MMMM'))
    COMMENT 'Jan, Feb,..., Dec',
  
  quarter tinyint NOT NULL GENERATED ALWAYS AS (CAST(quarter(date) AS TINYINT))
    COMMENT 'quarter number 1-4',
  
  year smallint NOT NULL GENERATED ALWAYS AS (
    CAST(year(date) AS SMALLINT)
    )
    COMMENT 'Gregorian year, not ISO',
  
  -- Period flags
  is_month_start BOOLEAN NOT NULL GENERATED ALWAYS AS (
    date = date_trunc('month', date)
    )
    COMMENT 'True if first day of month',

  is_month_end BOOLEAN NOT NULL GENERATED ALWAYS AS (
    date = last_day(date)
    )
    COMMENT 'True if last day of month',
  
  is_quarter_start BOOLEAN NOT NULL GENERATED ALWAYS AS (
    date = date_trunc('quarter', date)
    )
    COMMENT 'True if first day of quarter',
  
  is_quarter_end BOOLEAN NOT NULL GENERATED ALWAYS AS (
    date = last_day(date) AND quarter(date) <> quarter(date_add(date, 1))
    )
    COMMENT 'True if last day of quarter' 
)

USING DELTA
COMMENT 'Gold-Lite Date dimension (authoritative calendar ; one row per day)'
TBLPROPERTIES (
  delta.appendOnly = true
);

select * from main.retail_gold.datedim;

## Filling DateDim

In [0]:
%sql

-- Extended date range to encompase entire years of date bounds
WITH bounds AS (
  SELECT
    CAST('2011-01-01' AS DATE) AS d_min,
    CAST('2015-12-31' AS DATE) AS d_max
),
calendar AS (
  SELECT
    explode(sequence(BOUNDS.d_min, BOUNDS.d_max)) AS date
  FROM bounds
)

INSERT INTO main.retail_gold.DateDim(date)
SELECT
  c.date
FROM calendar c
LEFT ANTI JOIN main.retail_gold.datedim d ON c.date = d.date;

SELECT * from main.retail_gold.DatEDiM limit 10;

## Sanity checks on DateDim

In [0]:
%sql
-- Range: Expect min_date 2011-01-01, max_date 2015-12-31
SELECT min(date) AS min_date, max(date) AS max_date
FROM main.retail_gold.datedim;

In [0]:
%sql
-- Continuity: Expect no rows, datediff > 1 should evaluate to false in all cases.
WITH ordered AS (
  SELECT date, lag(date) OVER (ORDER BY date) AS prev_d
  FROM main.retail_gold.datedim
)
SELECT date AS gap_starts_after
FROM ordered
WHERE prev_d IS NOT NULL AND datediff(date, prev_d) > 1;

In [0]:
%sql
-- 3 Uniqueness of surrogate key: expect n = nd
SELECT COUNT(*) AS n, COUNT(DISTINCT date_sk) AS nd
FROM main.retail_gold.datedim;

# Step 2: Build ProductDim table

In [0]:
%sql
create or replace table main.retail_gold.ProductDim (
  product_id string not null
    comment 'Primary key. Unique identifier for product',
  
  product_name string not null
    comment 'The product\'s name',

  category string not null
    comment 'Top-level product grouping',
  
  subcategory string not null
    comment 'The product\'s sub-category. Rolls up to category',

  -- Hashing to quickly check for changes
  row_hash string generated always as (
    sha2(
      concat_ws('||',
        lower(TRIM(product_id)),
        lower(TRIM(product_name)),
        lower(TRIM(category)),
        lower(TRIM(subcategory))
      ),
      256
    )
  )
    comment "SHA2-256 hash of business attributes for change detection",

  -- Timestamps to track changes
  created_at timestamp not null default current_timestamp()
    comment 'Timestamp of creation',

  updated_at timestamp not null default current_timestamp()
    comment 'Timestamp of last update'
)

USING DELTA
COMMENT 'Gold-Lite Product dimension (Type-1; overwrite on change; one row per product)'
TBLPROPERTIES(
  'delta.feature.allowColumnDefaults' = 'supported'
  );

select * from main.retail_gold.ProductDim;

## Fill ProductDim table

In [0]:
%sql
MERGE INTO main.retail_gold.ProductDim AS t

USING (
  SELECT
    UPPER(TRIM(product_id)) AS product_id,
    TRIM(product_name)      AS product_name,
    TRIM(category)          AS category,
    TRIM(sub_category)      AS subcategory,
    ROW_NUMBER() OVER (
      PARTITION BY UPPER(TRIM(product_id))
      ORDER BY TRIM(product_name)
    ) AS rn,

    sha2(
      concat_ws('||',
        lower(trim(product_name)),
        lower(trim(sub_category)),
        lower(trim(category))
    ), 
    256
    ) AS incoming_hash
  FROM main.retail.superstore_silver
  WHERE product_id IS NOT NULL AND TRIM(product_id) <> ''
) s
ON t.product_id = s.product_id

/* Type-1 overwrite only when business attrs truly changed */
WHEN MATCHED AND s.rn = 1 AND t.row_hash <> s.incoming_hash THEN
  UPDATE SET
    product_name = s.product_name,
    category     = s.category,
    subcategory  = s.subcategory,
    updated_at   = current_timestamp()


WHEN NOT MATCHED AND s.rn = 1 THEN
  INSERT (product_id, product_name, category, subcategory)
  VALUES (s.product_id, s.product_name, s.category, s.subcategory);
